# Langchain Simple FAQ Chatbot

In [42]:
import os
from dotenv import load_dotenv

In [55]:
load_dotenv()

True

In [56]:
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"

---

### Load: Scape Data from Webpage

In [10]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [11]:
url='https://docs.langchain.com/langsmith/billing#understand-your-current-usage'

In [12]:
webbase_loader = WebBaseLoader(url)

In [13]:
webbase_loader

In [14]:
docs = webbase_loader.load()

In [15]:
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/billing#understand-your-current-usage', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content="Manage billing in your account - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesReleases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusOn this pageSet up billing for your accountDeveloper Plan: set up billing on

### Split - RecurCharSplitter

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [18]:
doc_chunks = text_splitter.split_documents(docs)

In [19]:
doc_chunks

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/billing#understand-your-current-usage', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content="Manage billing in your account - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesReleases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusOn this pageSet up billing for your accountDeveloper Plan: set up billing on

### Embed - OpenAIEmbeddings

In [20]:
from langchain_openai import OpenAIEmbeddings

In [21]:
openai_embeddings = OpenAIEmbeddings()

### Store - FAISS

In [22]:
from langchain_community.vectorstores import FAISS

In [23]:
vector_db = FAISS.from_documents(doc_chunks,openai_embeddings)

In [24]:
vector_db

---

### Query - .similarity_search()

In [25]:
query='Langsmith has 2 usage limits: total traces and extended'

In [26]:
query_results = vector_db.similarity_search(query)

In [27]:
query_results[0].page_content

'For organizations with multiple workspaces only: For simplicity, LangSmith incorporates the free traces into the cost calculation of the first workspace only. In actuality, the free traces can be “consumed” by any workspace. Therefore, although workspace-level spend limits are approximate for multi-workspace organizations, the organization-level spend limit is absolute.\n\u200bConfigure trace tier distrubution\nLangSmith has two trace tiers: base traces and extended traces. Base traces have the base retention and are short-lived (14 days), while extended traces have extended retention and are long-lived (400 days). For more information, refer to the data retention conceptual docs.'

### Query - Retrieval Chain

#### Setup LLM

In [31]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model='gpt-4o')

#### ChatPromptTemplate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [33]:
prompt=ChatPromptTemplate.from_template(
    """
    Answer the following questions based only on the provided context:

    <context>
        {context}
    </context>
    """
)

#### Document Chain

In [45]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [46]:
document_chain=create_stuff_documents_chain(llm,prompt)

In [47]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following questions based only on the provided context:\n\n    <context>\n        {context}\n    </context>\n    '), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resour

In [48]:
from langchain_core.documents import Document

In [ ]:
document_chain.invoke({
    "input":"Langsmith has 2 usage limits: total traces and extended",
    "context":[Document(page_content="Langsmith has 2 usage limits: total traces and extended traces. These correspond to the 2 metrices we've been tracking on our usage graph." )]
})

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


'Certainly! Based on the provided context:\n\n1. **What are the two usage limits of Langsmith?**\n   - Total traces\n   - Extended traces\n\n2. **What do these usage limits correspond to?**\n   - They correspond to the two metrics tracked on the usage graph.'

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


### Query - Retriever Chain

In [60]:
from langchain_classic.chains import create_retrieval_chain

In [58]:
vector_db

In [59]:
retriever = vector_db.as_retriever()

In [61]:
retriever_chain = create_retrieval_chain(retriever,document_chain)

In [62]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7a6c581e9550>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following questions based only on the provided context:\n\n    <context>\n        {context}\n    </context>\n    '), additional_kwargs={

In [63]:
retriever_response = retriever_chain.invoke({
    'input':'Langsmith has 2 usage limits: total traces and extended.'
})

In [64]:
retriever_response

{'input': 'Langsmith has 2 usage limits: total traces and extended.',
 'context': [Document(id='fff5e99e-fd74-4d6f-b92e-6fb22cbddabb', metadata={'source': 'https://docs.langchain.com/langsmith/billing#understand-your-current-usage', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='For organizations with multiple workspaces only: For simplicity, LangSmith incorporates the free traces into the cost calculation of the first workspace only. In actuality, the free traces can be “consumed” by any workspace. Therefore, although workspace-level spend limits are approximate for multi-workspace organizations, the organization-level spend limit is absolute.\n\u200bConfigure trace tier distrubution\nLangSmith has two trace tiers: base traces and extended traces. Base traces have the base retention and are short-lived (14 days), while extended traces have extended retention and are long-lived (400 days). For more information, refer to the data retentio

In [65]:
retriever_response['answer']

'1. **How are free traces incorporated in the cost calculation for organizations with multiple workspaces?**\n\n   For organizations with multiple workspaces, LangSmith incorporates the free traces into the cost calculation of the first workspace only. However, the free traces can actually be consumed by any workspace within the organization.\n\n2. **What are the retention periods for base traces and extended traces in LangSmith?**\n\n   Base traces have a retention period of 14 days, while extended traces have a retention period of 400 days.\n\n3. **How can a user set a spend limit for a LangSmith workspace?**\n\n   To set a spend limit for a workspace, a user should navigate to Settings -> Billing and Usage -> Usage limits, and input the desired spend limit for the selected workspace. LangSmith will then determine an appropriate number of base and extended trace limits to match that spend.\n\n4. **What happens when an automation rule matches any run within a trace in LangSmith?**\n\n

In [66]:
retriever_response['context']

[Document(id='fff5e99e-fd74-4d6f-b92e-6fb22cbddabb', metadata={'source': 'https://docs.langchain.com/langsmith/billing#understand-your-current-usage', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='For organizations with multiple workspaces only: For simplicity, LangSmith incorporates the free traces into the cost calculation of the first workspace only. In actuality, the free traces can be “consumed” by any workspace. Therefore, although workspace-level spend limits are approximate for multi-workspace organizations, the organization-level spend limit is absolute.\n\u200bConfigure trace tier distrubution\nLangSmith has two trace tiers: base traces and extended traces. Base traces have the base retention and are short-lived (14 days), while extended traces have extended retention and are long-lived (400 days). For more information, refer to the data retention conceptual docs.'),
 Document(id='92bf75d4-4a41-4c36-a870-7fa5d776a425', metadat

---